In [1]:
import pandas as pd
import numpy as np

In [2]:
dataretrieve = pd.read_csv("..//datasets/dataretrieve.csv", na_values='?')
desharnais = pd.read_csv("..//datasets/desharnais.csv", na_values='?')
dataretrieve = dataretrieve.fillna(0)
desharnais = desharnais.fillna(0)

In [3]:
with open("..//datasets/dataretrieve.txt") as infile:
    dataretrievelist = infile.read()
    dataretrievelist = dataretrievelist.split(',')
with open("..//datasets/desharnais.txt") as infile:
    desharnaislist = infile.read()
    desharnaislist = desharnaislist.split(',')

In [4]:
from sklearn.preprocessing import StandardScaler

In [5]:
dataretrieve.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9
0,127,126,6,7,2,10,1,120,0
1,458,441,50,67,17,23,1,391,0
2,182,178,10,14,3,13,1,168,0
3,270,270,14,14,3,10,1,256,0
4,107,104,11,14,4,21,1,93,0


In [10]:
features = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8']
x = dataretrieve.loc[:, features].values
y = dataretrieve.loc[:,['A9']].values

In [11]:
x = StandardScaler().fit_transform(x)

/Users/ajgan/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


In [13]:
pd.DataFrame(data = x, columns = features).head()

,A1,A2,A3,A4,A5,A6,A7,A8
0,-0.921917,-0.929818,-0.947340,-0.933219,-1.032295,-1.129954,-0.92582,-0.884712
1,-0.524600,-0.552801,-0.574992,-0.390367,-0.375104,-0.110401,-0.92582,-0.524117
2,-0.855898,-0.867580,-0.913491,-0.869886,-0.988483,-0.894673,-0.92582,-0.820843
3,-0.750267,-0.757468,-0.879641,-0.869886,-0.988483,-1.129954,-0.92582,-0.703749
4,-0.945924,-0.956149,-0.905028,-0.869886,-0.944670,-0.267256,-0.92582,-0.920639


In [17]:
mean_vec = np.mean(x, axis=0)

In [24]:
cov_mat = np.cov(x.T)
cov_mat

array([[ 1.00775194,  1.00619672,  0.73367136,  0.77403053,  0.81707477,
        -0.22460123, -0.0585032 ,  1.00327299],
       [ 1.00619672,  1.00775194,  0.7514519 ,  0.76955393,  0.82430483,
        -0.20742457, -0.05140755,  1.00220737],
       [ 0.73367136,  0.7514519 ,  1.00775194,  0.92683585,  0.89139059,
         0.31832215,  0.11964651,  0.67697755],
       [ 0.77403053,  0.76955393,  0.92683585,  1.00775194,  0.88044284,
         0.21530453,  0.07530098,  0.70981609],
       [ 0.81707477,  0.82430483,  0.89139059,  0.88044284,  1.00775194,
         0.11363766,  0.15077379,  0.77625447],
       [-0.22460123, -0.20742457,  0.31832215,  0.21530453,  0.11363766,
         1.00775194,  0.21285637, -0.28063849],
       [-0.0585032 , -0.05140755,  0.11964651,  0.07530098,  0.15077379,
         0.21285637,  1.00775194, -0.07592612],
       [ 1.00327299,  1.00220737,  0.67697755,  0.70981609,  0.77625447,
        -0.28063849, -0.07592612,  1.00775194]])

In [26]:
eig_vals, eig_vecs = np.linalg.eig(cov_mat)

print('Eigenvectors \n%s' %eig_vecs)
print('\nEigenvalues \n%s' %eig_vals)

Eigenvectors 
[[ -4.21310214e-01  -1.96919312e-01   6.36668852e-02  -2.87564610e-01
    9.11941285e-02  -9.51126918e-02   7.38938772e-01  -3.65486403e-01]
 [ -4.22778209e-01  -1.83646065e-01   6.04885435e-02  -3.08152302e-01
    3.20686902e-02   8.59491849e-02   9.82433976e-10   8.24924445e-01]
 [ -3.90319167e-01   2.90277365e-01  -1.54438349e-01   2.56113269e-01
    1.11184777e-01   8.04974268e-01  -1.38883736e-10  -1.16615444e-01]
 [ -3.96952986e-01   2.14192334e-01  -1.42687050e-01   4.36030162e-01
    5.63422926e-01  -5.06618621e-01  -9.80362959e-02   4.84679049e-02]
 [ -4.08119869e-01   1.57543105e-01   2.71311227e-02   3.20447977e-01
   -8.04637225e-01  -2.40321187e-01  -6.92625487e-14  -5.73195502e-05]
 [  9.05936583e-03   7.06173567e-01  -4.06276704e-01  -5.58901632e-01
   -6.01815481e-02  -1.42067614e-01   6.78070472e-15   5.94040563e-06]
 [ -1.40432286e-02   4.59850056e-01   8.79713975e-01  -7.04627354e-02
    9.72246437e-02   5.13872185e-03   3.93043120e-14   3.28216804e-05]

In [31]:
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

eig_pairs.sort()
eig_pairs.reverse()
eig_pairs

[(5.1939368050149852,
  array([-0.42131021, -0.42277821, -0.39031917, -0.39695299, -0.40811987,
          0.00905937, -0.01404323, -0.40864955])),
 (1.5837075087586956,
  array([-0.19691931, -0.18364607,  0.29027737,  0.21419233,  0.15754311,
          0.70617357,  0.45985006, -0.24978909])),
 (0.86520955428855328,
  array([ 0.06366689,  0.06048854, -0.15443835, -0.14268705,  0.02713112,
         -0.4062767 ,  0.87971397,  0.09156053])),
 (0.23340556858678035,
  array([-0.28756461, -0.3081523 ,  0.25611327,  0.43603016,  0.32044798,
         -0.55890163, -0.07046274, -0.38289621])),
 (0.11531580757475236,
  array([ 0.09119413,  0.03206869,  0.11118478,  0.56342293, -0.80463722,
         -0.06018155,  0.09722464,  0.01822824])),
 (0.070440141276629883,
  array([-0.09511269,  0.08594918,  0.80497427, -0.50661862, -0.24032119,
         -0.14206761,  0.00513872, -0.03092616])),
 (1.1837558503597492e-07,
  array([ -3.65486403e-01,   8.24924445e-01,  -1.16615444e-01,
           4.84679049e-0

In [41]:
matrix_w = np.hstack((eig_pairs[0][1].reshape(8,1), 
                      eig_pairs[1][1].reshape(8,1),
                      eig_pairs[2][1].reshape(8,1)))
matrix_w

array([[-0.42131021, -0.19691931,  0.06366689],
       [-0.42277821, -0.18364607,  0.06048854],
       [-0.39031917,  0.29027737, -0.15443835],
       [-0.39695299,  0.21419233, -0.14268705],
       [-0.40811987,  0.15754311,  0.02713112],
       [ 0.00905937,  0.70617357, -0.4062767 ],
       [-0.01404323,  0.45985006,  0.87971397],
       [-0.40864955, -0.24978909,  0.09156053]])

In [42]:
Y = x.dot(matrix_w)

In [43]:
Y

array([[  2.30733154,  -1.28790107,  -0.29986973],
       [  1.21338851,  -0.47757427,  -0.75010558],
       [  2.17300227,  -1.13184191,  -0.39471909],
       [  2.0187515 ,  -1.35843694,  -0.2802505 ],
       [  2.27365797,  -0.62049633,  -0.66996884],
       [  0.59591896,  -1.52363111,  -0.23318785],
       [  0.30548356,  -0.7245896 ,  -0.62703138],
       [  0.67019724,  -0.71544641,  -0.62937703],
       [ -2.44962359,  -3.29117239,   0.48138153],
       [ -4.39456581,  -1.87860159,  -0.18304536],
       [  0.01949885,  -0.61088534,  -0.67213426],
       [  2.08345743,  -1.46407604,  -0.21468878],
       [ -3.79165887,   0.84398252,  -1.51780009],
       [ -2.9949353 ,   1.15224241,  -1.485646  ],
       [ -0.12910073,   0.33489553,  -1.22855879],
       [ -3.51746119,   0.16608827,   1.04867328],
       [ -1.21966798,  -0.07137688,   1.24272477],
       [  0.41247157,   0.11911177,   1.17610413],
       [ -2.83554722,  -1.04313879,  -0.52130184],
       [ -0.14414185,  -0.08646